In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

In [2]:
# MNIST 数据集相关的常数设置
INPUT_NODE = 784

OUTPUT_NODE = 10

LAYER1_NODE = 500  
# 隐藏层的节点数（本结构只有一个隐藏层）

BATCH_SIZE = 100

LRARNING_RATE_BASE = 0.8

LRARNING_RATE_DECAY = 0.99  
# 学习率的衰减率

REGULARIZATION_RATE = 0.0001  
# 正则化项在损失函数中的系数

TRAINNING_STEPS = 30000  
# 训练轮数

MOVING_AVERAGE_DECAY = 0.99  
# 滑动平均衰减率

MODEL_SAVE_PATH = r"E:\Code\DeepLearning\TrainDemo\01 MNIST\TensorFlow\Data\checkPoint"
MODEL_NAME = "model.ckpt"

In [3]:
# 辅助函数，计算神经网络的前向传播结果
# 定义了一个生成变量的函数
def get_weight_variable(shape, regularizer):
    weights = tf.get_variable(
        "weights", shape,
        initializer=tf.truncated_normal_initializer(stddev=0.1)
    )

    if regularizer != None:
        tf.add_to_collection('losses', regularizer(weights))
    return weights

In [4]:
# 神经网络前向传播的过程
def inference(input_tensor, regularizer):
    # 声明第一层神经网络
    with tf.variable_scope('layer1'):
        weights = get_weight_variable([INPUT_NODE, LAYER1_NODE], regularizer)
        biases = tf.get_variable("biases", [LAYER1_NODE],initializer=tf.truncated_normal_initializer(stddev=0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights)) + biases

    # 声明第二层神经网络
    with tf.variable_scope('layer2'):
        weights = get_weight_variable([LAYER1_NODE, OUTPUT_NODE], regularizer)
        biases = tf.get_variable("biases", [OUTPUT_NODE], initializer=tf.truncated_normal_initializer(stddev=0.0))
        layer2 = tf.matmul(layer1, weights) + biases

    return layer2

In [5]:
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')

    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y = inference(x, regularizer)
    global_step = tf.Variable(0, trainable=False)

    # 损失函数、学习率、滑动平均、训练过程
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.arg_max(y_, 1), logits=y)
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(LRARNING_RATE_BASE, global_step, mnist.train.num_examples / BATCH_SIZE,
                                               LRARNING_RATE_DECAY)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')

    # 初始化TensorFlow持久化类
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        for i in range(TRAINNING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x:xs, y_:ys})
            if i % 1000 == 0:
                print('After %d training step(s), loss on training batch is %g' % (step, loss_value))
                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)

In [6]:
def main(argv = None):
    mnist = input_data.read_data_sets(r"E:\DataSet\MNIST", one_hot=True)
    train(mnist)

if __name__ == '__main__':
    tf.app.run()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting E:\DataSet\MNIST\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting E:\DataSet\MNIST\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting E:\DataSet\MNIST\t10k-images-idx3-ubyte.gz
Extracting E:\DataSet\MNIST\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Use `argmax` instead
After 1 training step(s), loss on training batch is 3.01492
After 1001 training step(s), loss on training batch is 0.331485
After 2001 training step(s), loss on training batch is 0.180675
After 3001 training step(s), loss on trai

SystemExit: 

C:\Users\Echo\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3273: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
